In [ ]:
import pandas as pd
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
Renewable_Energy = "Solar_PBE"

try:
    address = "/content/drive/My Drive/Doctor/Research/DRL/Colab/"
    data_train_csv1 = pd.read_csv(address+Renewable_Energy+'_16.csv', index_col=0)

except:
    address = ""
    data_train_csv1 = pd.read_csv(address+Renewable_Energy+'_16.csv', index_col=0)

data_train_csv2 = pd.read_csv(address+Renewable_Energy+'_17.csv', index_col=0)
data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
data_val_csv    = pd.read_csv(address+Renewable_Energy+'_18.csv', index_col=0)
data_test_csv   = pd.read_csv(address+Renewable_Energy+'_19.csv', index_col=0)

train_predict = np.array(pd.read_csv(address+"results/"+Renewable_Energy+"_Model1_train.csv", index_col=0))
val_predict = np.array(pd.read_csv(address+"results/"+Renewable_Energy+"_Model1_val.csv", index_col=0))
test_predict = np.array(pd.read_csv(address+"results/"+Renewable_Energy+"_Model1_NEC.csv", index_col=0))

In [ ]:
# Data Preprocessing

Battery_Size = 0.5
unit         = 4 #unit: 15 minute

RE_Capacity1 = max(data_train_csv['Power(MW)'])
RE_Capacity2 = max(data_val_csv['Power(MW)'])
RE_Capacity3 = max(data_test_csv['Power(MW)'])

size_train0 = int(len(data_train_csv)/unit)
size_val0   = int(len(data_val_csv)/unit)
size_test0  = int(len(data_test_csv)/unit)

data_train0 = []
data_train  = []
for i in range(size_train0):
    data_train0 += [round(pd.Series.mean(data_train_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity1, 3)]
    data_train  += [data_train0[i]] if data_train0[i] > 0 else []

data_val0 = []
data_val  = []
for i in range(size_val0):
    data_val0 += [round(pd.Series.mean(data_val_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity2, 3)]
    data_val  += [data_val0[i]] if data_val0[i] > 0 else []

data_test0 = []
data_test  = []
for i in range(size_test0):
    data_test0 += [round(pd.Series.mean(data_test_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity3, 3)]
    data_test  += [data_test0[i]] if data_test0[i] > 0 else []

In [ ]:
size_train = len(data_train)
size_val = len(data_val)
size_test = len(data_test)

train_input = np.zeros((size_train-1, 1))
train_output = np.zeros((size_train-1, 1))
for i in range(size_train-1):
    train_input[i,:] = data_train[i]
    train_output[i,:] = data_train[i+1]

val_input = np.zeros((size_val-1, 1))
val_output = np.zeros((size_val-1, 1))
for i in range(size_val-1):
    val_input[i,:] = data_val[i]
    val_output[i,:] = data_val[i+1]

test_input = np.zeros((size_test-1, 1))
test_output = np.zeros((size_test-1, 1))
for i in range(size_test-1):
    test_input[i,:] = data_test[i]
    test_output[i,:] = data_test[i+1]

print("MAPE_train: {}%".format(round(100*np.mean(np.abs(train_predict - train_output)/train_output),2)).ljust(25), end="")
print("MAPE_val: {}%".format(round(100*np.mean(np.abs(val_predict - val_output)/val_output),2)).ljust(25), end="")
print("MAPE_test: {}%".format(round(100*np.mean(np.abs(test_predict - test_output)/test_output),2)).ljust(25))

MAPE_train: 36.56%       MAPE_val: 36.96%         MAPE_test: 36.16%        


In [ ]:
# SARSA

in_size       = 1
out_size      = 5
gamma         = 0.99
epsilon       = 0.9
batch_size    = 128
learning_rate = 0.001

class SARSA(nn.Module):
    def __init__(self):
        super(SARSA, self).__init__()
        self.fc = nn.Linear(in_size, out_size)

    def forward(self, x):
        x = self.fc(x)
        return x
        
def train_net(model, batch, optimizer):
    s_lst, a_lst, r_lst, s_prime_lst = [], [], [], []

    for transition in batch:
        s, a, r, s_prime = transition
        s_lst.append(s)
        a_lst.append([a])
        r_lst.append([r])
        s_prime_lst.append(s_prime)

    s,a,r,s_prime = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                    torch.tensor(r_lst, dtype=torch.float), torch.tensor(s_prime_lst, dtype=torch.float)
            
    for i in range(10):
        q_out = model.forward(s).gather(1,a)
        target = r + gamma * torch.mean(model.forward(s_prime))
        loss = F.mse_loss(q_out, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
# Training SARSA

E_max   = Battery_Size
tdelta  = unit/4
eff_c   = 0.9
eff_d   = 0.9
soc_min = 0.1
soc_max = 0.9
P_cmax  = Battery_Size/3
P_dmax  = Battery_Size/3
beta_c  = 0.01
beta_d  = 0.01

E_cmax = eff_c*P_cmax*tdelta
E_dmax = (1/eff_d)*P_dmax*tdelta
C_max  = int(out_size/2)

total_episode = 20
print_interval = 1

model = SARSA()
act_train,  act_val,  act_test  = [], [], [] # Controlled Value
mape_train, mape_val, mape_test = [], [], [] # Mean Absolute Percentage Error
ccr_train,  ccr_val,  ccr_test  = [], [], [] # Complete Compensation Ratio

batch = collections.deque(maxlen=batch_size+1)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for n_epi in range(total_episode):
    act_train  += [[]]; act_val  += [[]]; act_test  += [[]]
    mape_train += [[]]; mape_val += [[]]; mape_test += [[]]
    ccr_train  += [[]]; ccr_val  += [[]]; ccr_test  += [[]]

    state = [E_max/2]
    i = 0
    while i < size_train-1:
        for t in range(batch_size):
            coin = torch.rand(1).item()
            if coin < epsilon:
                action = np.random.choice(range(out_size))
            else:
                Qout = model.forward(torch.tensor(state, dtype=torch.float))
                action = np.argmax(Qout.tolist())
            E_c = min(max((E_cmax/C_max)*(action-C_max), 0.0), E_max*soc_max - state[0])
            E_d = min(max((E_dmax/C_max)*(C_max-action), 0.0), state[0] - E_max*soc_min)

            real = train_output[i][0] + eff_d*E_d/tdelta - (1/eff_c)*E_c/tdelta
            pred = train_predict[i][0]

            E = state[0] + E_c - E_d
            P_climit = min(P_cmax, (1/eff_c)*(E_max*soc_max - E)/tdelta)
            P_dlimit = min(P_dmax, eff_d*(E - E_max*soc_min)/tdelta)
            P_c = min(max(real-pred, 0), P_climit)
            P_d = min(max(pred-real, 0), P_dlimit)
            E_prime = E + eff_c*P_c*tdelta - (1/eff_d)*P_d*tdelta
            disp = 0.0 if np.isclose(real - P_c + P_d, 0) else real - P_c + P_d
            error = pred - disp
            error_function = abs(error) + beta_c*P_c + beta_d*P_d

            next_state = [E_prime]
            reward = -error_function
            batch.append((state, action, reward, next_state))
            state = next_state[:]

            act_train[n_epi]  += [E_c - E_d]
            mape_train[n_epi]  += [abs((pred-disp)/disp)] if disp != 0 else [0]
            ccr_train[n_epi] += [1 if np.isclose(pred-disp,0) else 0]
            i += 1
            if i == size_train-1:
                break

        if n_epi != 0:
            train_net(model, batch, optimizer)
    
    state = [E_max/2]
    for k in range(size_val-1):
        Qout = model.forward(torch.tensor(state, dtype=torch.float))
        action = np.argmax(Qout.tolist())
        E_c = min(max((E_cmax/C_max)*(action-C_max), 0.0), E_max*soc_max - state[0])
        E_d = min(max((E_dmax/C_max)*(C_max-action), 0.0), state[0] - E_max*soc_min)

        real = val_output[k][0] + eff_d*E_d/tdelta - (1/eff_c)*E_c/tdelta
        pred = val_predict[k][0]

        E = state[0] + E_c - E_d
        P_climit = min(P_cmax, (1/eff_c)*(E_max*soc_max - E)/tdelta)
        P_dlimit = min(P_dmax, eff_d*(E - E_max*soc_min)/tdelta)
        P_c = min(max(real-pred, 0), P_climit)
        P_d = min(max(pred-real, 0), P_dlimit)
        E_prime = E + eff_c*P_c*tdelta - (1/eff_d)*P_d*tdelta
        disp = 0.0 if np.isclose(real - P_c + P_d, 0) else real - P_c + P_d
        error = pred - disp
        error_function = abs(error) + beta_c*P_c + beta_d*P_d

        next_state = [E_prime]
        state = next_state[:]

        act_val[n_epi]  += [E_c - E_d]
        mape_val[n_epi]  += [abs((pred-disp)/disp)] if disp != 0 else [0]
        ccr_val[n_epi] += [1 if np.isclose(pred-disp,0) else 0]
    
    state = [E_max/2]
    for l in range(size_test-1):
        Qout = model.forward(torch.tensor(state, dtype=torch.float))
        action = np.argmax(Qout.tolist())
        E_c = min(max((E_cmax/C_max)*(action-C_max), 0.0), E_max*soc_max - state[0])
        E_d = min(max((E_dmax/C_max)*(C_max-action), 0.0), state[0] - E_max*soc_min)

        real = test_output[l][0] + eff_d*E_d/tdelta - (1/eff_c)*E_c/tdelta
        pred = test_predict[l][0]

        E = state[0] + E_c - E_d
        P_climit = min(P_cmax, (1/eff_c)*(E_max*soc_max - E)/tdelta)
        P_dlimit = min(P_dmax, eff_d*(E - E_max*soc_min)/tdelta)
        P_c = min(max(real-pred, 0), P_climit)
        P_d = min(max(pred-real, 0), P_dlimit)
        E_prime = E + eff_c*P_c*tdelta - (1/eff_d)*P_d*tdelta
        disp = 0.0 if np.isclose(real - P_c + P_d, 0) else real - P_c + P_d
        error = pred - disp
        error_function = abs(error) + beta_c*P_c + beta_d*P_d

        next_state = [E_prime]
        state = next_state[:]

        act_test[n_epi]  += [E_c - E_d]
        mape_test[n_epi]  += [abs((pred-disp)/disp)] if disp != 0 else [0]
        ccr_test[n_epi] += [1 if np.isclose(pred-disp,0) else 0]
    
    if (n_epi+1)%print_interval == 0:
        MAPE_train  = round(100*np.mean(mape_train[n_epi]),2)
        MAPE_val    = round(100*np.mean(mape_val[n_epi]),2)
        MAPE_test   = round(100*np.mean(mape_test[n_epi]),2)
        CCR_train = round(np.mean(ccr_train[n_epi]),3)
        CCR_val   = round(np.mean(ccr_val[n_epi]),3)
        CCR_test  = round(np.mean(ccr_test[n_epi]),3)

        print("episode: {}".format(n_epi+1))
        print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
        print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
        print("MAPE_test: {}%".format(MAPE_test).ljust(25))
        print("CCR_train: {}".format(CCR_train).ljust(25), end="")
        print("CCR_val: {}".format(CCR_val).ljust(25), end="")
        print("CCR_test: {}".format(CCR_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

episode: 1
MAPE_train: 23.57%       MAPE_val: 67.27%         MAPE_test: 89.77%        
CCR_train: 0.685         CCR_val: 0.482           CCR_test: 0.456          
------------------------------------------------------------------------------------------
episode: 2
MAPE_train: 30.5%        MAPE_val: 57.25%         MAPE_test: 68.27%        
CCR_train: 0.695         CCR_val: 0.609           CCR_test: 0.622          
------------------------------------------------------------------------------------------
episode: 3
MAPE_train: 29.04%       MAPE_val: 2.77%          MAPE_test: 2.58%         
CCR_train: 0.705         CCR_val: 0.81            CCR_test: 0.825          
------------------------------------------------------------------------------------------
episode: 4
MAPE_train: 23.14%       MAPE_val: 3.49%          MAPE_test: 3.3%          
CCR_train: 0.715         CCR_val: 0.812           CCR_test: 0.826          
---------------------------------------------------------------------------

In [ ]:
# Produce results
#
#select_num = np.argmin(np.mean(mape_val,axis=1))
#select = pd.DataFrame(np.array(act_test[select_num][:]))
#select.to_csv(address+Renewable_Energy+"_Model2_ECC+_"+str(int(100*E_max))+".csv")